# Edosoft - Resumen y clasificación de textos

In [1]:
import numpy as np
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
import string
import math

# Imports
import pandas as pd

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.es import Spanish

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

import pickle

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer

import PyPDF2
from tika import parser

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
file = open("prueba_es.txt", "r", encoding='utf8')
lines = file.readlines()

datos = {'Documentos':lines}
df = pd.DataFrame(data = datos)
df.head(10)

In [ ]:
df['Documentos'][0]

In [ ]:
# ### Primero tokenizamos los 4 primeros documentos
# def tokenize_words(text):
#     sin_puntuacion = text.translate(str.maketrans('', '', string.punctuation))
#     tokenized_text=word_tokenize(sin_puntuacion)
#     print(tokenized_text)
#     print(type(tokenized_text))
    
#     return tokenized_text

# # df_tokenized = df.copy()
# df['Documentos'] = df['Documentos'].apply(lambda x: tokenize_words(x))
    


In [ ]:
# stopword_es = nltk.corpus.stopwords.words('spanish')
# #from nltk and spacy library
# total_spanish_stopwords = list(stopwords.words("spanish")) + stopword_es

# def remove_stopwords(text):
#     text = [word for word in text if word not in stopword_es]
#     return text

# df['Sin_stopwords'] = df['Documentos'].apply(lambda x: remove_stopwords(x))

In [ ]:
# df['Sin_stopwords'][0]

In [ ]:
# def spanish_stemmer(x):
#     spanish_stemmer_ = SnowballStemmer(language = 'spanish')
#     for i in range(len(x)):
# #         print(i)
#         for l in range(len(x[i])):
# #             print(l)
#             x[i][l] = spanish_stemmer_.stem(x[i][l])
# #             print(x[i])
#     return x
    
# df['words_stemmed'] = spanish_stemmer(df['Sin_stopwords'])

In [ ]:
# df['words_stemmed'][0]

## Empezamos a hacer el resumen

Trying with the following guide: [Click on here](https://towardsdatascience.com/text-summarization-using-tf-idf-e64a0644ace3)

In [ ]:
df['Documentos'][0]

In [ ]:
sentences = sent_tokenize(df['Documentos'][0]) # NLTK function
total_documents = len(sentences)

stopword_es = nltk.corpus.stopwords.words('spanish')
#from nltk and spacy library
total_spanish_stopwords = list(stopwords.words("spanish")) + stopword_es

In [ ]:
# sentences

In [2]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    
    stopword_es = nltk.corpus.stopwords.words('spanish')
    #from nltk and spacy library
    total_spanish_stopwords = list(stopwords.words("spanish")) + stopword_es
    stopWords = total_spanish_stopwords
    ps = SnowballStemmer(language = 'spanish')

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table


def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix


def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue


def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average


def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [ ]:
freq_matrix= _create_frequency_matrix(sentences)

tf_matrix = _create_tf_matrix(freq_matrix)

count_doc_per_words = _create_documents_per_words(freq_matrix)

idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)

tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)

sentence_scores = _score_sentences(tf_idf_matrix)

threshold = _find_average_score(sentence_scores)

In [ ]:
threshold

In [ ]:
summary = _generate_summary(sentences, sentence_scores, 1* threshold)
print(summary)

In [ ]:
len(summary)

In [ ]:
len(df['Documentos'][0])

In [ ]:
df.columns

In [ ]:
file_2 = open("boe.txt", "r", encoding='utf8')
lines_2 = file_2.readlines()


df = pd.DataFrame(data=[lines,lines_2],columns=['Documento'])
df['Documento'][1]

In [ ]:
df['Documento'][1]

In [ ]:
file_2

In [ ]:
with open('boe.txt', encoding='Latin1') as f:
    contents = f.read()
    print(contents)

##### https://www.simplifiedpython.net/pdf-to-text-python-extract-text-from-pdf-documents-using-pypdf2-module/



In [ ]:
pdfFileObject = open(r"prueba_es.pdf", 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
print(" No. Of Pages :", pdfReader.numPages)

In [ ]:
pageObject = pdfReader.getPage(0)
 
print(pageObject.extractText())

In [ ]:
texto = pdfReader.getPage(0).extractText() + pdfReader.getPage(1).extractText()

In [ ]:
texto.replace('\n',' ').lstrip().rstrip()

In [ ]:

texto = " ".join(texto.replace('\n',' ').split())

In [ ]:
# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file

from tika import parser

raw = parser.from_file('prueba_es.pdf')
print(raw['content'])


In [ ]:
" ".join(raw['content'].split())

In [ ]:
# https://www.tutorialspoint.com/working-with-pdf-files-in-python

# with open('BOE-A-2010-4132.pdf', 'rb') as f:
#     pdf = PyPDF2.PdfFileReader(f)
#     info = pdf.getDocumentInfo()
#     number_of_pages = pdf.getNumPages()

# print("Author: \t", info.author)
# print()
# print("Creator: \t", info.creator)
# print()
# print("Producer: \t",info.producer)
# print()
# print("Subject: \t", info.subject)
# print()
# print("title: \t",info.title)
# print()
# print("Number of Pages in pdf: \t",number_of_pages)

# prueba = ''
# for i in range(pdf.getNumPages()):
#     prueba = prueba + pdf.getPage(i).extractText()
#     print(len(prueba))
    
# pdf.getPage(2)

In [ ]:
# pdf.getPage(2).extractText()

In [ ]:
with open('BOE-A-2010-4132.pdf', 'rb') as f:
    pdf = PyPDF2.PdfFileReader(f)
    info = pdf.getDocumentInfo()
    number_of_pages = pdf.getNumPages()
print("Author: \t", info.author)
print()
print("Creator: \t", info.creator)
print()
print("Producer: \t",info.producer)
print()
print("Subject: \t", info.subject)
print()
print("title: \t",info.title)
print()
print("Number of Pages in pdf: \t",number_of_pages)

In [ ]:
# pdfFileObject = open('BOE-A-2010-4132.pdf', 'rb')
# pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
# prueba = ''
# for i in range(pdfReader.getNumPages()):
#     prueba = prueba + pdfReader.getPage(i).extractText()
#     print(len(prueba))
# prueba = prueba.replace('\n',' ')

In [ ]:
# pdfReader.getPage(1).extractText()

In [ ]:
# prueba

In [ ]:
pdfReader.getDocumentInfo()

In [ ]:
documento = 'BOE-A-2010-4132.pdf'

with open(documento, 'rb') as f:
    pdf = PyPDF2.PdfFileReader(f)
    info = pdf.getDocumentInfo()
    number_of_pages = pdf.getNumPages()

# text = ''
# for i in range(pdf.getNumPages()):
#     text = text + pdf.getPage(i).extractText()
#     print(len(prueba))

In [9]:
# Incluir el documento en PDF
# --------------------------------------------------------------------

def resume_pdfs(documento, alpha):
    
    #### First parser of PDFs files
    # ==============================================================================
    
    pdfFileObject = open(documento, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
#     text = ''
#     for i in range(pdfReader.getNumPages()):
#         text = text + pdfReader.getPage(i).extractText()
#     text = text.replace('\n',' ')
#     print(text)
#         print(len(prueba))
      
    
    #### Second parser of PDFs files
    # ==============================================================================
    raw = parser.from_file(documento)
#     print(raw['content'])

    text = " ".join(raw['content'].split())

    # -----------------------------------
    
    
    sentences = sent_tokenize(text)
    total_documents = len(sentences)
    #print(sentences)

    # 2 Create the Frequency matrix of the words in each sentence.
    freq_matrix = _create_frequency_matrix(sentences)
    #print(freq_matrix)

    '''
    Term frequency (TF) is how often a word appears in a document, divided by how many words are there in a document.
    '''
    # 3 Calculate TermFrequency and generate a matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    '''
    Inverse document frequency (IDF) is how unique or rare a word is.
    '''
    # 5 Calculate IDF and generate a matrix
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(tf_idf_matrix)
    #print(sentence_scores)

    # 8 Find the threshold
    threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary (alpha between 0.5 - 1.5)
    summary = _generate_summary(sentences, sentence_scores, alpha * threshold)
    
    
    print("Título: \t",pdfReader.getDocumentInfo().title)
    print('--------------------------------------------------------------------')
    print("Número de páginas: {} \nLongitud del documento original: {}\nLongitud del resumen: {}".format(pdfReader.getNumPages(), len(text), len(summary)))
    print('--------------------------------------------------------------------')
    print(summary)
    
    return summary

In [11]:
# resume_pdfs('prueba_es.pdf', 0.8)
resume = resume_pdfs('BOE-A-2010-4132.pdf', 0.5)

Título: 	 Disposición 4132 del BOE núm. 62 de 2010
--------------------------------------------------------------------
Número de páginas: 10 
Longitud del documento original: 33619
Longitud del resumen: 8502
--------------------------------------------------------------------
 Disposición 4132 del BOE núm. 62 de 2010 BOLETÍN OFICIAL DEL ESTADO Núm. 62 Viernes 12 de marzo de 2010 Sec. I. Pág. cv e: B O E -A -2 01 0- 41 32 BOLETÍN OFICIAL DEL ESTADO Núm. 62 Viernes 12 de marzo de 2010 Sec. I. Pág. Objeto. 1. Los requisitos mínimos se referirán a la titulación académica del profesorado, la relación numérica alumno-profesor, las instalaciones docentes y deportivas y el número de puestos escolares. 2. Artículo 2. Denominación de los centros docentes. Artículo 3. Requisitos de instalaciones comunes a todos los centros. 1. 2. b) Reunir las condiciones de seguridad estructural, de seguridad en caso de incendio, de seguridad de utilización, de salubridad, de protección frente al ruido y de aho

In [ ]:
# import textract
# PDF_read = textract.process('BOE-A-2010-4132.pdf', method='pdfminer')

In [ ]:
# from pdfminer.high_level import extract_text
# PDF_read = extract_text('prueba_es.pdf')

In [ ]:
# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file
# Para instalar java runtime en anaconda: https://anaconda.org/cyclus/java-jre
from tika import parser

raw = parser.from_file('prueba_es.pdf')
print(raw['content'])

texto_2 = " ".join(raw['content'].split())
texto_2

# Pruebas para otro proyecto

In [ ]:
a = 'tes'

In [ ]:
#https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8
# https://stackoverflow.com/questions/56955320/file-contain-u00c2-u00a0-convert-to-characters

import json
# load data using Python JSON module
with open('jsonformatter.json','r', encoding=['Latin1']) as f:
    data = json.loads(f.read())
# Flatten data
data_json = pd.json_normalize(data)

In [ ]:
# https://stackoverflow.com/questions/56955320/file-contain-u00c2-u00a0-convert-to-characters
# https://jsonformatter.org/json-parser
with open("jsonformatter.json", encoding='utf8') as f:
    data = json.loads(f.read().encode('Latin1').decode('utf8'))
    print(data[2])

In [ ]:
data

In [ ]:
data_json

In [ ]:
data_json_formatted_1 = data_json.to_json(orient="split")
data_json_formatted_2 = data_json.to_json(orient="records")

In [ ]:
# with open('data_json_formatted.json', 'w', encoding='utf-8') as f:
#     json.dump(data_json_formatted, f, ensure_ascii=False, indent=4)

with open('data_json_formatted_split.json', 'w') as f:
    f.write(data_json_formatted_1)
0
with open('data_json_formatted_records.json', 'w') as f:
    f.write(data_json_formatted_2)

In [ ]:
data_json_formatted_2